In [2]:
from astropy.io import fits
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import seaborn as sns
from scipy import stats
import matplotlib.mlab as mlab
from scipy.optimize import curve_fit
import os
from astropy.wcs import WCS
from photutils import DAOStarFinder
from astropy.stats import mad_std
from photutils import aperture_photometry, CircularAperture, CircularAnnulus
from area import area
%matplotlib inline
sns.set()

ModuleNotFoundError: No module named 'photutils'

In [ ]:
NNser_bias_files = os.listdir('C:/Users/seanj/3rd year labs/astro analysis/data_2019/NNser/bias/')
x = len(NNser_bias_files)

NNser_bias_directories = []
for i in range(x):
    NNser_bias_directories.append(f'C:/Users/seanj/3rd year labs/astro analysis/data_2019/NNser/bias/{NNser_bias_files[i]}')

bias_data = np.zeros(shape=(2048, 2098))
for file in NNser_bias_directories:
    image = fits.open(file)
    bias_data += (image[0].data)

NNser_master_bias = bias_data/len(NNser_bias_directories)

In [ ]:
NNser_flat_files = os.listdir('C:/Users/seanj/3rd year labs/astro analysis/data_2019/NNser/flats/clear/')
x = len(NNser_flat_files)

NNser_flat_directories = []
for i in range(x):
    NNser_flat_directories.append(f'C:/Users/seanj/3rd year labs/astro analysis/data_2019/NNser/flats/clear/{NNser_flat_files[i]}')

flat_data = np.zeros(shape=(2048, 2098))
for file in NNser_flat_directories:
    image = fits.open(file)
    flat_data += ((image[0].data) - NNser_master_bias)

NNser_master_flats = flat_data/len(NNser_flat_directories)
NNser_master_flats_normalised = NNser_master_flats/np.mean(NNser_master_flats)

In [ ]:
# in this cell we are comparing the difference between reduced and non-reduced images
NNser_object_clear = os.listdir('C:/Users/seanj/3rd year labs/astro analysis/data_2019/NNser/object/clear/')
x = len(NNser_object_clear)

NNser_object_clear_directories = []
for i in range(x):
    NNser_object_clear_directories.append(f'C:/Users/seanj/3rd year labs/astro analysis/data_2019/NNser/object/clear/{NNser_object_clear[i]}')


image = fits.open(NNser_object_clear_directories[1])
image1 = (image[0].data-NNser_master_bias)/NNser_master_flats_normalised

#here I removed the 50 left most columns because they contained a lot of objects(outliers)
#and dont matter to the region of interest
image1 = image1[:,50:] 

print(image[0].header['ST'])
print(image[0].header['DATE-OBS'])
print(image[0].header['EXPTIME'])


In [ ]:
# in this section i used the DAOStarFinder function to highlight all
# the objects and then picked out from the list the locations of refrence stars and NN Sers 

sigma = mad_std(image1)

daofind = DAOStarFinder(fwhm=6, threshold=4*sigma)
sources = daofind(image1)

fig= plt.figure(figsize=(15,13))
plt.scatter(sources['xcentroid'], sources['ycentroid'], alpha=0.3, color="green")
plt.imshow(image1, vmin=2000, vmax=4000)
plt.colorbar()
plt.show()

In [ ]:
# here i was just using the DAOStarFinder inorder to accurately determine the location of the refrence stars
xcentroids_cal = [1338.4475620048397,1294.8859844473714,1067.579209977523,1379.4417361202306]
ycentroids_cal = [1150.7320661531699,1497.2093184654789,1481.902428352763,839.0148119303864]

xcentroid_NN = 1446.14785188 
ycentroid_NN = 1145.7259363

fig= plt.figure(figsize=(10,8))
plt.imshow(image1, vmin=2000, vmax=4000)
plt.colorbar()
plt.xlim(800,1600)
plt.ylim(800,1600)
plt.scatter(xcentroids_cal,ycentroids_cal, alpha=0.5, color="green")
plt.scatter(xcentroid_NN,ycentroid_NN, alpha=0.5, color="green")
plt.show()

fig= plt.figure(figsize=(10,8))
plt.imshow(image1, vmin=2000, vmax=4000)
plt.colorbar()
plt.scatter(xcentroids_cal,ycentroids_cal, alpha=0.5, color="green")
plt.scatter(xcentroid_NN,ycentroid_NN, alpha=0.5, color="green")
plt.show()

In [ ]:
#defining the refrence star magnitudes, taken from figure 4
refrence_mag = [15.8,15.1,13.7,13,7]

# Get the positions of sources in the field from the table above.
positions = np.transpose((xcentroids_cal,ycentroids_cal))

# Set up aperture and annulus
aperture = CircularAperture(positions, r=6.)
annulus_aperture = CircularAnnulus(positions, r_in=10., r_out=15.)

# Make a list of apertures
apers = [aperture, annulus_aperture]

#define the areas of the apertures
aperture_area = np.pi*6**2
annulus_aperture_area = np.pi*15**2 - np.pi*10**2

ZP_list = []

for image in NNser_object_clear_directories:
    image1 = fits.open(image)
    image2 = (image1[0].data-NNser_master_bias)/NNser_master_flats_normalised
    image2 = image2[:,50:]
    phot_table = aperture_photometry(image2, apers)

    # We calculate the mean counts in each pixel in the background annulus, and then multiply by the area
    # in the aperture to get the total background counts within each aperture
    bkg_mean = phot_table['aperture_sum_1']/annulus_aperture_area 
    bkg_sum = bkg_mean * aperture_area

    # Now we get the final table of background subtracted counts within each aperture
    final_sum = phot_table['aperture_sum_0'] - bkg_sum
    #print(final_sum)
    
    list = []
    for i in range(4):
        ZP = 2.5*np.log10(final_sum[i]) + refrence_mag[i]
        list.append(ZP)
        
    ZP_list.append(np.mean(list))

print(ZP_list)

In [ ]:
#magnitudes of NN Ser using the calculated ZPs

position = np.transpose((xcentroid_NN,ycentroid_NN))

aperture = CircularAperture(position, r=6.)
annulus_aperture = CircularAnnulus(position, r_in=10., r_out=15.)

apers = [aperture, annulus_aperture]


final_sum_list = []
times = []
exposure_times = []
F_star = []
F_sky = []

for image in NNser_object_clear_directories:
    image1 = fits.open(image)
    image2 = (image1[0].data-NNser_master_bias)/NNser_master_flats_normalised
    image2 = image2[:,50:]
    phot_table = aperture_photometry(image2, apers)
    time = image1[0].header['ST']
    exposure_time = image1[0].header['EXPTIME']
    exposure_times.append(exposure_time)
    times.append(time)

    bkg_mean = phot_table['aperture_sum_1']/annulus_aperture_area 
    bkg_sum = bkg_mean * aperture_area
    
    F_star.append(phot_table['aperture_sum_0'])
    F_sky.append(phot_table['aperture_sum_1'])

    final_sum = phot_table['aperture_sum_0'] - bkg_sum
    final_sum_list.append(final_sum[0])
    
    

NN_magnitudes = []
for i in range(18):
    magtitude = -2.5*np.log10(final_sum_list[i]) + ZP_list[0]
    NN_magnitudes.append(magtitude)


print(NN_magnitudes)
print(times)
print(exposure_times)

In [ ]:
R =  3.37
n_star = aperture_area   #one unit^2 is eqaul to one pixel
n_sky = annulus_aperture_area

magnitude_error_list = []

for i in range(18):
    signal_noise = F_star[i]/np.sqrt(F_star[i] + n_star*(1+n_star/n_sky)*(F_sky[i]+R**2))
    magnitude_error = 2.5*np.log10(1+ 1/signal_noise)
    magnitude_error_list.append(magnitude_error[0])
    
print(magnitude_error_list)

In [ ]:
#limiting factor calculation
positions_all = np.transpose((sources['xcentroid'], sources['ycentroid']))
positions_limit = np.transpose((802,918))

aperture = CircularAperture(positions_all, r=6.)
annulus_aperture = CircularAnnulus(positions_all, r_in=10., r_out=15.)

apers_limit = [aperture, annulus_aperture]

final_sum_list =[]

image = fits.open(NNser_object_clear_directories[7])
image = (image[0].data-NNser_master_bias)/NNser_master_flats_normalised
image = image[:,50:]

phot_table = aperture_photometry(image, apers_limit)

bkg_mean = phot_table['aperture_sum_1']/annulus_aperture_area 
bkg_sum = bkg_mean * aperture_area

final_sum = phot_table['aperture_sum_0'] - bkg_sum

limiting_magnitudes = []
for i in range(65):
    magnitude = -2.5*np.log10(final_sum[i]) + ZP_list[7]
    if np.isnan(magnitude) == True:
        continue
    else:
        limiting_magnitudes.append(magnitude)
    
limiting_magnitude = np.max(limiting_magnitudes)
print(limiting_magnitude)

In [ ]:
def get_sec(time_h):
    times_s = []
    times_s1 = []
    for i in range(len(time_h)):
        h, m, s = (time_h[i]).split(':')
        times_s.append(int(h) * 3600 + int(m) * 60 + int(s))
    for i in range(len(times_sec)):
        times_s1.append(times_sec[i] - times_sec[0])
    return times_s1

times_sec =get_sec(times)

fig= plt.figure(figsize=(15,10))
plt.plot(times_sec[:6],NN_magnitudes[:6], color="blue",linewidth = '2',alpha = 0.5)
plt.plot(times_sec[10:],NN_magnitudes[10:], color="blue",linewidth = '2',alpha = 0.5)
plt.errorbar(times_sec,NN_magnitudes,yerr=magnitude_error_list, xerr=None, fmt='.', color="red",label='NN Ser magnitudes')
#error bars are almost too small to see
plt.axhline(y = limiting_magnitude , color = 'black', label='Limiting magnitude')

plt.axvline(x = times_sec[5] , color = 'green', linestyle = '--', label = 'Outer limits of the eclipse')
plt.axvline(x = times_sec[10] , color = 'green', linestyle = '--')
plt.axvline(x = times_sec[6] , color = 'purple', linestyle = '--', label = 'Inner limits of the eclipse')
plt.axvline(x = times_sec[9] , color = 'purple', linestyle = '--')

plt.xlabel("Time (seconds)")
plt.ylabel("Brightness (magnitude)")

print(times[5])
print(times[10])
print(times[6])
print(times[9])

print(times_sec[10]-times_sec[5])
print(times_sec[9]-times_sec[6])

plt.ylim(22, 16)
plt.grid(True)
plt.legend()
plt.show()